In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [164]:
import tensorflow as tf
print(tf.__version__)
tf.keras.backend.clear_session()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Bidirectional,GRU,Dense,Dropout,Activation,Embedding
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import re
import string
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import nltk
nltk.download('popular')


2.1.0
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!

True

In [ ]:
!wget "https://www.awesomefilm.com/script/tomorrowneverdies.txt"
corpus=open('/kaggle/working/tomorrowneverdies.txt','rb',).readlines()
corp=[str(i).lower() for i in corpus]

In [ ]:
def regex_clean(text):
    out=re.sub(r"\\+[nt]", ' ', text,0, re.MULTILINE)
    out=re.sub(r"""[()-\.+:;\\'"b\\]""", ' ', out,0, re.MULTILINE)  
    return out

stop=set(stopwords.words('english'))
stem=PorterStemmer()
punct=string.punctuation
def clean(text):
    out=regex_clean(text)
    words=word_tokenize(out)
    temp=''
    for i in words:
        if i not in stop:
            if i not in punct:
                temp+=i+' '
    return temp


cleaned_text=[clean(i) for i in corp]
cleaned_text=[i for i in cleaned_text if i!='']


words=[]
for sent in cleaned_text:
    for word in sent.split():
        words.append(word)
count=Counter(words)


text=[]
for sent in cleaned_text:
    temp=[]
    for word in sent.split():
        if count[word]>2:
            temp.append(word)
    if len(temp)>=4:
        text.append(temp)
        
maxlen=0
for sent in text:
    if len(sent)>maxlen:
        maxlen=len(sent)
        
tokenizer=Tokenizer()
tokenizer.fit_on_texts(text)
sequences=tokenizer.texts_to_sequences(text)


vocab_size=len(tokenizer.word_counts)
n_sequence=np.empty((len(sequences),4),dtype=np.float32)
for i in range(len(sequences)):
    n_sequence[i]=sequences[i][:4]
    
    
train_seq=n_sequence[:,:-1]
target_seq=n_sequence[:,-1]
# train_seq
target_seq=to_categorical(target_seq,num_classes=vocab_size+1)
target_seq.shape,train_seq.shape

In [ ]:
unique_words=set()
for i in cleaned_text:
    for j in i.split():
        unique_words.add(j)
print(len(unique_words),unique_words)


In [163]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model,self).__init__()
        self.embed=Embedding(vocab_size+1,1028)
        self.d1=LSTM(512,return_sequences=True)
        self.d2=LSTM(256,return_sequences=True)
        self.d3=LSTM(128)
        self.d4=Dense(64,activation='relu')
        self.classifier=Dense(vocab_size+1,activation='softmax')
        
    def call(self,input):
        out=self.embed(input)
        out=self.d1(out)
        out=self.d2(out)
        out=self.d3(out)
        out=self.d4(out)
        out=self.classifier(out)
#         print(out.numpy())
        return out
with tf.device('/device:GPU:0'):
    model=Model()
    model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.sparse_categorical_crossentropy)
    model.fit(train_seq,target_seq,epochs=500,batch_size=128)
        

In [ ]:
embeddings_index = dict()
f = open('/kaggle/input/glove6b100dtxt/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()

In [ ]:
embedding_matrix = np.zeros((vocab_size+1, 100))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [171]:
class Bidirectional_LSTM_Model(tf.keras.Model):
    def __init__(self):
        super(Bidirectional_LSTM_Model,self).__init__()
        self.embed=Embedding(vocab_size+1,100,weights=[embedding_matrix],trainable=True)
        self.d1=Bidirectional(LSTM(512,return_sequences=True))
        self.d2=Bidirectional(LSTM(256,return_sequences=True))
        self.d3=Bidirectional(LSTM(128))
        self.d4=Dense(64,activation='relu')
        self.classifier=Dense(vocab_size+1,activation='softmax')
        
    def call(self,input):
        out=self.embed(input)
        out=self.d1(out)
        out=self.d2(out)
        out=self.d3(out)
        out=self.d4(out)
        out=self.classifier(out)
        return out


In [172]:
class LSTM_Model(tf.keras.Model):
    def __init__(self):
        super(LSTM_Model,self).__init__()
        self.embed=Embedding(vocab_size+1,100,weights=[embedding_matrix],trainable=True)
        self.d1=LSTM(512,return_sequences=True)
        self.d2=LSTM(256,return_sequences=True)
        self.d3=LSTM(128)
        self.d4=Dense(64,activation='relu')
        self.classifier=Dense(vocab_size+1,activation='softmax')
        
    def call(self,input):
        out=self.embed(input)
        out=self.d1(out)
        out=self.d2(out)
        out=self.d3(out)
        out=self.d4(out)
        out=self.classifier(out)
        return out

In [173]:
class GRU_Model(tf.keras.Model):
    def __init__(self):
        super(GRU_Model,self).__init__()
        self.embed=Embedding(vocab_size+1,100,weights=[embedding_matrix],trainable=True)
        self.d1=GRU(512,return_sequences=True)
        self.d2=GRU(256,return_sequences=True)
        self.d3=GRU(128)
        self.d4=Dense(64,activation='relu')
        self.classifier=Dense(vocab_size+1,activation='softmax')
        
    def call(self,input):
        out=self.embed(input)
        out=self.d1(out)
        out=self.d2(out)
        out=self.d3(out)
        out=self.d4(out)
        out=self.classifier(out)
        return out

In [174]:
class Bidirectional_GRU_Model(tf.keras.Model):
    def __init__(self):
        super(Bidirectional_GRU_Model,self).__init__()
        self.embed=Embedding(vocab_size+1,100,weights=[embedding_matrix],trainable=True)
        self.d1=Bidirectional(GRU(512,return_sequences=True))
        self.d2=Bidirectional(GRU(256,return_sequences=True))
        self.d3=Bidirectional(GRU(128))
        self.d4=Dense(64,activation='relu')
        self.classifier=Dense(vocab_size+1,activation='softmax')
        
    def call(self,input):
        out=self.embed(input)
        out=self.d1(out)
        out=self.d2(out)
        out=self.d3(out)
        out=self.d4(out)
        out=self.classifier(out)
        return out

In [180]:
with tf.device('/device:GPU:0'):
    model1=LSTM_Model()
    model1.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.sparse_categorical_crossentropy)
    model1.fit(train_seq,target_seq,epochs=500,batch_size=64)
    model2=GRU_Model()
    model2.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.sparse_categorical_crossentropy)
    model2.fit(train_seq,target_seq,epochs=500,batch_size=64)
    model3=Bidirectional_LSTM_Model()
    model3.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.sparse_categorical_crossentropy)
    model3.fit(train_seq,target_seq,epochs=500,batch_size=64)
    model4=Bidirectional_GRU_Model()
    model4.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.sparse_categorical_crossentropy)
    model4.fit(train_seq,target_seq,epochs=500,batch_size=64)

Train on 1850 samples
Epoch 1/500
1850/1850 [==============================] - 4s 2ms/sample - loss: 7.1618
Epoch 2/500
1850/1850 [==============================] - 0s 151us/sample - loss: 6.5751
Epoch 3/500
1850/1850 [==============================] - 0s 151us/sample - loss: 6.3907
Epoch 4/500
1850/1850 [==============================] - 0s 152us/sample - loss: 6.3405
Epoch 5/500
1850/1850 [==============================] - 0s 156us/sample - loss: 6.3131
Epoch 6/500
1850/1850 [==============================] - 0s 144us/sample - loss: 6.2670
Epoch 7/500
1850/1850 [==============================] - 0s 195us/sample - loss: 6.2147
Epoch 8/500
1850/1850 [==============================] - 0s 212us/sample - loss: 6.1675
Epoch 9/500
1850/1850 [==============================] - 0s 165us/sample - loss: 6.0962
Epoch 10/500
1850/1850 [==============================] - 0s 229us/sample - loss: 5.9710
Epoch 11/500
1850/1850 [==============================] - 0s 167us/sample - loss: 5.8191
Epoch 12/5

Epoch 93/500
1850/1850 [==============================] - 0s 163us/sample - loss: 0.0773
Epoch 94/500
1850/1850 [==============================] - 0s 154us/sample - loss: 0.0747
Epoch 95/500
1850/1850 [==============================] - 0s 151us/sample - loss: 0.0788
Epoch 96/500
1850/1850 [==============================] - 0s 150us/sample - loss: 0.0801
Epoch 97/500
1850/1850 [==============================] - 0s 156us/sample - loss: 0.0869
Epoch 98/500
1850/1850 [==============================] - 0s 159us/sample - loss: 0.0900
Epoch 99/500
1850/1850 [==============================] - 0s 151us/sample - loss: 0.1720
Epoch 100/500
1850/1850 [==============================] - 0s 157us/sample - loss: 0.9735
Epoch 101/500
1850/1850 [==============================] - 0s 154us/sample - loss: 2.1830
Epoch 102/500
1850/1850 [==============================] - 0s 152us/sample - loss: 1.5318
Epoch 103/500
1850/1850 [==============================] - 0s 150us/sample - loss: 0.7196
Epoch 104/500
185

1850/1850 [==============================] - 0s 162us/sample - loss: 0.0502
Epoch 185/500
1850/1850 [==============================] - 0s 164us/sample - loss: 0.0499
Epoch 186/500
1850/1850 [==============================] - 0s 164us/sample - loss: 0.0512
Epoch 187/500
1850/1850 [==============================] - 0s 164us/sample - loss: 0.0511
Epoch 188/500
1850/1850 [==============================] - 0s 168us/sample - loss: 0.0489
Epoch 189/500
1850/1850 [==============================] - 0s 164us/sample - loss: 0.0491
Epoch 190/500
1850/1850 [==============================] - 0s 164us/sample - loss: 0.0485
Epoch 191/500
1850/1850 [==============================] - 0s 161us/sample - loss: 0.0495
Epoch 192/500
1850/1850 [==============================] - 0s 225us/sample - loss: 0.0503
Epoch 193/500
1850/1850 [==============================] - 0s 203us/sample - loss: 0.0487
Epoch 194/500
1850/1850 [==============================] - 0s 168us/sample - loss: 0.0462
Epoch 195/500
1850/1850 

1850/1850 [==============================] - 0s 205us/sample - loss: 0.0453
Epoch 276/500
1850/1850 [==============================] - 0s 160us/sample - loss: 0.0459
Epoch 277/500
1850/1850 [==============================] - 0s 155us/sample - loss: 0.0463
Epoch 278/500
1850/1850 [==============================] - 0s 164us/sample - loss: 0.0451
Epoch 279/500
1850/1850 [==============================] - 0s 159us/sample - loss: 0.0449
Epoch 280/500
1850/1850 [==============================] - 0s 170us/sample - loss: 0.0470
Epoch 281/500
1850/1850 [==============================] - 0s 162us/sample - loss: 0.0453
Epoch 282/500
1850/1850 [==============================] - 0s 162us/sample - loss: 0.0472
Epoch 283/500
1850/1850 [==============================] - 0s 164us/sample - loss: 0.0470
Epoch 284/500
1850/1850 [==============================] - 0s 154us/sample - loss: 0.0455
Epoch 285/500
1850/1850 [==============================] - 0s 165us/sample - loss: 0.0447
Epoch 286/500
1850/1850 

1850/1850 [==============================] - 0s 154us/sample - loss: 0.0420
Epoch 367/500
1850/1850 [==============================] - 0s 171us/sample - loss: 0.0430
Epoch 368/500
1850/1850 [==============================] - 0s 150us/sample - loss: 0.0424
Epoch 369/500
1850/1850 [==============================] - 0s 155us/sample - loss: 0.0438
Epoch 370/500
1850/1850 [==============================] - 0s 222us/sample - loss: 0.0420
Epoch 371/500
1850/1850 [==============================] - 0s 152us/sample - loss: 0.0398
Epoch 372/500
1850/1850 [==============================] - 0s 150us/sample - loss: 0.0420
Epoch 373/500
1850/1850 [==============================] - 0s 143us/sample - loss: 0.0425
Epoch 374/500
1850/1850 [==============================] - 0s 149us/sample - loss: 0.0408
Epoch 375/500
1850/1850 [==============================] - 0s 144us/sample - loss: 0.0407
Epoch 376/500
1850/1850 [==============================] - 0s 151us/sample - loss: 0.0411
Epoch 377/500
1850/1850 

...

1850/1850 [==============================] - 0s 228us/sample - loss: 0.0528
Epoch 51/500
1850/1850 [==============================] - 0s 220us/sample - loss: 0.0517
Epoch 52/500
1850/1850 [==============================] - 0s 236us/sample - loss: 0.0496
Epoch 53/500
1850/1850 [==============================] - 0s 230us/sample - loss: 0.0501
Epoch 54/500
1850/1850 [==============================] - 0s 229us/sample - loss: 0.0498
Epoch 55/500
1850/1850 [==============================] - 0s 235us/sample - loss: 0.0490
Epoch 56/500
1850/1850 [==============================] - 0s 227us/sample - loss: 0.0501
Epoch 57/500
1850/1850 [==============================] - 0s 227us/sample - loss: 0.0500
Epoch 58/500
1850/1850 [==============================] - 0s 222us/sample - loss: 0.0495
Epoch 59/500
1850/1850 [==============================] - 0s 236us/sample - loss: 0.0517
Epoch 60/500
1850/1850 [==============================] - 0s 224us/sample - loss: 0.0481
Epoch 61/500
1850/1850 [==========

1850/1850 [==============================] - 0s 219us/sample - loss: 0.0430
Epoch 142/500
1850/1850 [==============================] - 0s 223us/sample - loss: 0.0424
Epoch 143/500
1850/1850 [==============================] - 0s 223us/sample - loss: 0.0443
Epoch 144/500
1850/1850 [==============================] - 0s 220us/sample - loss: 0.0429
Epoch 145/500
1850/1850 [==============================] - 0s 234us/sample - loss: 0.0430
Epoch 146/500
1850/1850 [==============================] - 0s 218us/sample - loss: 0.0429
Epoch 147/500
1850/1850 [==============================] - 0s 229us/sample - loss: 0.0426
Epoch 148/500
1850/1850 [==============================] - 0s 234us/sample - loss: 0.0425
Epoch 149/500
1850/1850 [==============================] - 0s 234us/sample - loss: 0.0424
Epoch 150/500
1850/1850 [==============================] - 0s 227us/sample - loss: 0.0413
Epoch 151/500
1850/1850 [==============================] - 1s 355us/sample - loss: 0.0429
Epoch 152/500
1850/1850 

1850/1850 [==============================] - 0s 234us/sample - loss: 0.0399
Epoch 233/500
1850/1850 [==============================] - 0s 232us/sample - loss: 0.0385
Epoch 234/500
1850/1850 [==============================] - 0s 226us/sample - loss: 0.0385
Epoch 235/500
1850/1850 [==============================] - 0s 227us/sample - loss: 0.0393
Epoch 236/500
1850/1850 [==============================] - 0s 225us/sample - loss: 0.0395
Epoch 237/500
1850/1850 [==============================] - 0s 222us/sample - loss: 0.0418
Epoch 238/500
1850/1850 [==============================] - 1s 283us/sample - loss: 0.0380
Epoch 239/500
1850/1850 [==============================] - 0s 244us/sample - loss: 0.0386
Epoch 240/500
1850/1850 [==============================] - 0s 220us/sample - loss: 0.0389
Epoch 241/500
1850/1850 [==============================] - 0s 229us/sample - loss: 0.0384
Epoch 242/500
1850/1850 [==============================] - 0s 217us/sample - loss: 0.0378
Epoch 243/500
1850/1850 

1850/1850 [==============================] - 0s 223us/sample - loss: 0.0373
Epoch 324/500
1850/1850 [==============================] - 0s 227us/sample - loss: 0.0372
Epoch 325/500
1850/1850 [==============================] - 0s 235us/sample - loss: 0.0368
Epoch 326/500
1850/1850 [==============================] - 0s 228us/sample - loss: 0.0369
Epoch 327/500
1850/1850 [==============================] - 0s 233us/sample - loss: 0.0374
Epoch 328/500
1850/1850 [==============================] - 0s 228us/sample - loss: 0.0368
Epoch 329/500
1850/1850 [==============================] - 0s 256us/sample - loss: 0.0366
Epoch 330/500
1850/1850 [==============================] - 0s 240us/sample - loss: 0.0368
Epoch 331/500
1850/1850 [==============================] - 0s 242us/sample - loss: 0.0369
Epoch 332/500
1850/1850 [==============================] - 0s 236us/sample - loss: 0.0369
Epoch 333/500
1850/1850 [==============================] - 0s 232us/sample - loss: 0.0371
Epoch 334/500
1850/1850 

1850/1850 [==============================] - 0s 228us/sample - loss: 0.0369
Epoch 415/500
1850/1850 [==============================] - 1s 291us/sample - loss: 0.0367
Epoch 416/500
1850/1850 [==============================] - 0s 220us/sample - loss: 0.0363
Epoch 417/500
1850/1850 [==============================] - 0s 237us/sample - loss: 0.0363
Epoch 418/500
1850/1850 [==============================] - 0s 229us/sample - loss: 0.0365
Epoch 419/500
1850/1850 [==============================] - 0s 227us/sample - loss: 0.0363
Epoch 420/500
1850/1850 [==============================] - 0s 233us/sample - loss: 0.0366
Epoch 421/500
1850/1850 [==============================] - 0s 232us/sample - loss: 0.0365
Epoch 422/500
1850/1850 [==============================] - 0s 234us/sample - loss: 0.0370
Epoch 423/500
1850/1850 [==============================] - 0s 228us/sample - loss: 0.0375
Epoch 424/500
1850/1850 [==============================] - 0s 223us/sample - loss: 0.0369
Epoch 425/500
1850/1850 

In [186]:
def gen_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    output_text_1,output_2,output_3,output_4 = [],[],[],[]
    input_text = seed_text
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len,truncating='pre')
        pred_word_ind_1 = np.argmax(model1.predict(pad_encoded,verbose=0)[0])
        
        pred_word = tokenizer.index_word[pred_word_ind_1]
        input_text += ' '+pred_word
        output_text_1.append(pred_word)
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len,truncating='pre')
        pred_word_ind_2 = np.argmax(model2.predict(pad_encoded,verbose=0)[0])

        pred_word = tokenizer.index_word[pred_word_ind_2]
        input_text += ' '+pred_word
        output_2.append(pred_word)
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len,truncating='pre')
        pred_word_ind_3 = np.argmax(model3.predict(pad_encoded,verbose=0)[0])

        pred_word = tokenizer.index_word[pred_word_ind_3]
        input_text += ' '+pred_word
        output_3.append(pred_word)
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len,truncating='pre')
        pred_word_ind_4 = np.argmax(model4.predict(pad_encoded,verbose=0)[0])

        pred_word = tokenizer.index_word[pred_word_ind_4]
        input_text += ' '+pred_word
        output_4.append(pred_word)
        return output_text_1,output_2,output_3,output_4


print('\n\n===>Enter --exit to exit from the program')
while True:
    seed_text  = input('Enter string: ')
    if seed_text.lower() == '--exit':
        print('==============Stopped================')
        break
    else:
        out1,out2,out3,out4 = gen_text(model, tokenizer, seq_len=3, seed_text=seed_text, num_gen_words=50)
        print(f'''Output from LSTM : {seed_text} ==> {' '.join([i for i in out1])}''')
        print('\n')

        print(f'''Output from GRU : {seed_text} ==> {' '.join([i for i in out2])}''')
        print('\n')

        print(f'''Output from Bidirectional LSTM : {seed_text} ==> {' '.join([i for i in out3])}''')
        print('\n')

        print(f'''Output from Bidirectional GRU : {seed_text} ==> {' '.join([i for i in out4])}''')
        
        print('\n')




===>Enter --exit to exit from the program
Enter string: james bond
Output from LSTM : james bond ==> elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot


Output from GRU : james bond ==> elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot elliot


Output from Bidirectional LSTM : james bond ==> gra ond tank pulls area core starting har nuclear nuclear power intake part seal lights night johnstone near rep

In [187]:
model1.save_weights('saved_LSTM_weights.h5')

In [190]:
!git init
!apt-get install git-lfs

Reinitialized existing Git repository in /kaggle/working/.git/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 50 not upgraded.
Need to get 2129 kB of archives.
After this operation, 7662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2129 kB]
Fetched 2129 kB in 1s (2559 kB/s) 
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package git-lfs.
(Reading database ... 110879 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...


In [194]:
!git lfs install
!git lfs track ".h5"
!git add .gitattributes
!git config --global user.email "vncvish@gmail.com"
!git config --global user.name "vishal-nath-chauhan"
!git add saved_LSTM_weights.h5
!git commit -m 'saved lstm model weights'

Updated git hooks.
Git LFS initialized.
".h5" already supported
[master (root-commit) 5c553cd] saved lstm model weights
 2 files changed, 1 insertion(+)
 create mode 100644 .gitattributes
 create mode 100644 saved_LSTM_weights.h5


In [195]:
!git remote add origin https://github.com/vishal-nath-chauhan/james_bond_quadgram.git

In [ ]:
!git push -u origin master

Username for 'https://github.com': 